In [171]:
import pickle
import random
import time
import numpy as np

from tensorflow.python.keras.layers import RepeatVector

from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout

### Зададим расположение файлов

In [172]:
dataset_path = "D:/Temp/Dataset/kaggle/flickr_30k" # основной путь к датасету
end_dir = dataset_path + "/copy_flickr30k_images"

path_tokenizer = dataset_path + "/ru-tokenizer-train.pkl"
path_train_dict = dataset_path + "/captions-ru-train.pkl"
path_val_dict = dataset_path + "/captions-ru-val.pkl"

path_features_vgg16 = "features.pkl"

## Загрузка данных

In [173]:
def image_names_set(data):
    vals = set()

    for idx in data.index:
        vals.add(data.iat[idx, 0][:-4])

    return vals

def load_image_features(filename, data):
    with open (filename, 'rb') as f:
        all_features = pickle.load(f)
    features = {k: all_features[k] for k in data}

    return features

# Токенизируем данные

Подпись будет генерироваться по одному слову за раз. Для того чтобы знать первое и последнее слова
мы добавили в предыдущем файле к каждому описанию слова start и end

Далее текст подписи кодируется в число (токенизируется)

In [182]:
def to_lines(data):
    all_vals = list()
    print(data.keys())
    for key in data.keys():
        [all_vals.append(d) for d in data[key]]

    return all_vals

def create_tokenizer(data):
    lines = to_lines(data)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)

    return tokenizer

def find_max_words(data):
    lines = to_lines(data)
    return max(len(l.split()) for l in lines)

### Создание последовательности


In [175]:
def create_sequences(tokenizer, max_words, captions_list, image_name):
    X_image, X_text, y_word = list(), list(), list()
    vocab_size = len(tokenizer.word_index) + 1

    for caption in captions_list:
        seq = tokenizer.texts_to_sequences([caption])[0]

        for i in range(1, len(seq)):
            in_seq, out_seq = seq[:i], seq[i]
            in_seq = pad_sequences([in_seq], maxlen=max_words)[0]
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

            X_image.append(image_name)
            X_text.append(in_seq)
            y_word.append(out_seq)

    return X_image, X_text, y_word

### Генератор данных
Генератор данных будет выдавать данные на одно изображении в каждой партии. Это будут все последовательности, сгенерированные для изображения и её набора описаний.
Функция data_generator() будет генератором данных и будет принимать загруженные текстовые описания, признаки изображений, токенизатор и максимальную длину.

In [176]:
def data_generator(tokenizer, max_words, data, images, batch_size, random_seed):
    count = 0
    random.seed(random_seed)

    img_names = list(data.keys())
    assert batch_size <= len(img_names), 'batch size must be less than or equal to {}'.format(len(img_names))

    while True:
        input_img_batch, input_seq_batch, output_word_batch = list(), list(), list()

        if count >= len(img_names):
            count = 0
        start_i = count
        end_i = min(len(img_names), count + batch_size)

        for i in range(start_i, end_i):
            curr_img = img_names[i]
            image = images[curr_img][0]
            captions_list = data[curr_img]
            random.shuffle(captions_list)

            input_img, input_seq, output_word = create_sequences(tokenizer, max_words, captions_list, image)

            for j in range(len(input_img)):
                input_img_batch.append(input_img[j])
                input_seq_batch.append(input_seq[j])
                output_word_batch.append(output_word[j])

        count = count + batch_size
        yield [np.array(input_img_batch), np.array(input_seq_batch)], np.array(output_word_batch)

### Построение модели

In [185]:
from keras.layers import Bidirectional, TimeDistributed, add


def build_rnn(input_size, vocab_size, max_words):
    inputs1 = Input(shape=(input_size,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    inputs2 = Input(shape=(max_words,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)

    de1 = add([fe2, se3])
    de2 = Dense(256, activation='relu')(de1)
    outputs = Dense(vocab_size, activation='softmax')(de2)

    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

def build_alt_rnn(input_size, vocab_size, max_words):
    image_input = Input(shape=(input_size,))
    fe1 = Dense(256, activation='relu')(image_input)
    image_model = RepeatVector(max_words)(fe1)

    caption_input = Input(shape=(max_words,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(caption_input)
    se2 = LSTM(256, return_sequences=True)(se1)
    caption_model = TimeDistributed(Dense(256))(se2)

    de1 = add([image_model, caption_model])
    de2 = Bidirectional(LSTM(256, return_sequences=False))(de1)
    final_model = Dense(vocab_size, activation='softmax')(de2)

    model = Model(inputs=[image_input, caption_input], outputs=final_model)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

### Обучение
Отбросим загрузку тестового набора данных и контрольные точки модели и просто сохраним модель после каждой эпохи обучения. Затем мы сможем вернуться и загрузить/оценить каждую сохраненную модель после обучения, чтобы найти ту, которая имеет наименьшие потери.

In [194]:
def load_train_data(train_dict_path, tokenizer_path, features_path):
    with open (train_dict_path, 'rb') as f:
        out_train_dict = pickle.load(f)
    print('кол-во подписей .............. %d' % len(out_train_dict))

    out_train_features = load_image_features(features_path, out_train_dict)

    with open (tokenizer_path, 'rb') as f:
        out_tokenizer = pickle.load(f)
    out_vocab_size = len(out_tokenizer.word_index) + 1
    print('размер словаря ............... %d' % out_vocab_size)

    out_max_words = find_max_words(out_train_dict)
    print('длина предложения в словах ... %d' % out_max_words)

    return out_train_dict, out_tokenizer, out_vocab_size, out_max_words, out_train_features

def train_model(model, train_dict, tokenizer, max_words, train_features, batch_size, epochs_num):
    steps = len(train_dict)/batch_size
    if len(train_dict) % batch_size != 0:
        steps = steps + 1

    start_time = time.time()
    for i in range(epochs_num):
        generator = data_generator(tokenizer, max_words, train_dict, train_features, batch_size, 42)
        model.fit(generator,
                  epochs=1, steps_per_epoch=steps,
                  verbose=1)
        model.save('model-' + str(i) + '.h5')

    time_difference = time.time() - start_time
    minutes = time_difference/60
    print('время обучения в минутах ..... %d' % minutes)

### Набор для обучения

In [195]:
batch_size = 16
epochs_num = 20
train_dict, tokenizer, vocab_size, max_words, train_features = load_train_data(path_train_dict, path_tokenizer, path_features_vgg16)

кол-во подписей .............. 22247
размер словаря ............... 38126
длина предложения в словах ... 57


## Обучение VGG16

In [196]:
model = build_rnn(4096, vocab_size, max_words)
train_model(model, train_dict, tokenizer, max_words, train_features, batch_size, epochs_num)

1391/1391 [==============================] - 3499s 3s/step - loss: 2.7467
время обучения в минутах ..... 981


## Обучение модифицированной VGG16

In [197]:
model = build_alt_rnn(4096, vocab_size, max_words)
train_model(model, train_dict, tokenizer, max_words, train_features, batch_size, epochs_num)

 128/1391 [=>............................] - ETA: 3:29:27 - loss: 2.5988


KeyboardInterrupt

